# Instance 9

In [1]:
import os
from pyomo.environ import *
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

## Importing DataSet

In [2]:
file_path = 'Instance_9.xlsx'
# Read the Excel file into a dictionary of DataFrames, where keys are sheet names
Instance_data = pd.read_excel(file_path, sheet_name=None)

In [3]:
# Exam Data Set

Exams = Instance_data['Sheet1']
Exams['Exams'] = Exams['Exams'] - 1
ExamsList = list(Exams['Exams'])

### Dictionary of Students and their related Exams

In [4]:
# Creating a Dictionary of Students and Their Related Exams
Students_exams = Instance_data['Sheet3']
Students_exams['ExamNumber'] = Students_exams['ExamNumber'] - 1
# Initialize an empty dictionary to store the students and their exams
students = {}
# Iterate over the rows in the DataFrame
for index, row in Students_exams.iterrows():
    student = row['Student']
    exam_number = row['ExamNumber']
    
    # Check if the student is already in the dictionary
    if student in students:
        students[student].append(exam_number)
    # If the student is not in the dictionary, create a new key-value pair    
    else:
        students[student] = [exam_number]

### TimeSlots

In [5]:
Time_s = Instance_data['Sheet2']
time_slots = Time_s.loc[0, 'TimeSlot']
TimeList= list(range (time_slots))

In [6]:
TimeList

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]

### Distance

In [7]:
#Distance
Distance=list(range(1,6))
Distance

[1, 2, 3, 4, 5]

###  Finding Conflicted Exams (Weighted-Unweighted)

In [8]:
# Unweighted
# Step 1: Initialize the dictionary to store conflicting exams
conflicting_exams = {exam: set() for exams_list in students.values() for exam in exams_list}

# Step 2: Populate the dictionary with conflicting exams
for student_exams in students.values():
    for i, exam1 in enumerate(student_exams):
        for exam2 in student_exams[i + 1:]:
            conflicting_exams[exam1].add(exam2)
            conflicting_exams[exam2].add(exam1)

# Step 3: Create the unweighted conflicting matrix
Unweighted_Conflict_Matrix = [
    [1 if exam2 in conflicting_exams[exam1] else 0 for exam2 in ExamsList] for exam1 in ExamsList
]


In [9]:
# Weighted

# The weighted Conflicting MATRIX
# Step 1: Initialize the dictionary to store conflicting exams
conflicting_exams = {exam: set() for exams_list in students.values() for exam in exams_list}

# Step 2: Populate the dictionary with conflicting exams
for student_exams in students.values():
    for i, exam1 in enumerate(student_exams):
        for exam2 in student_exams[i + 1:]:
            conflicting_exams[exam1].add(exam2)
            conflicting_exams[exam2].add(exam1)

# Step 3: Create the weighted conflicting matrix
weighted_conflicting_matrix = [[0 for _ in ExamsList] for _ in ExamsList]

# Step 4: Calculate the number of students in each conflicting exam pair and fill the matrix
for i, exam1 in enumerate(ExamsList):
    for j, exam2 in enumerate(ExamsList):
        if exam2 in conflicting_exams[exam1]:
            count = sum(1 for student_exams in students.values() if exam1 in student_exams and exam2 in student_exams)
            weighted_conflicting_matrix[i][j] = count



### Converting the Llist of UC and WC to Matrix of Coefficients

In [10]:
#What We have:
#Exams
#TimeSlots
#Distance
#Conflicting Exams (Weighted and Unweighted)
#aij

#nij
#aij=UC_Matrix and nij=WC_Matrix
UC_Matrix=np.array(Unweighted_Conflict_Matrix )
WC_Matrix=np.array(weighted_conflicting_matrix)

In [11]:
UC_Matrix

array([[0, 1, 0, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [12]:
WC_Matrix

array([[ 0,  5,  0, ...,  0,  0,  0],
       [ 5,  0, 10, ...,  0,  0,  0],
       [ 0, 10,  0, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0]])

In [13]:
# Setting up the environment


env = gp.Env(
    r"C:\Users\ghadi\Downloads\Discrete Optimization Project - Gurobi- Instance 9\Discrete Optimization Project - Instance 9-Log File.log")
env.start()

# Environment parameters
env.setParam("Threads", 1)  # Controls the number of threads to apply to parallel algorithms
env.setParam("Presolve", 1)  # Controls the presolve level. conservative (1).
env.setParam("MIPGap", 1e-4)
env.setParam('Method', 0)  # Algorithm used to solve the initial root relaxation of a MIP model. 0=primal simplex.
env.setParam("TimeLimit", 1200)  # 20 minutes time limit
env.setParam("PreSparsify", 1) # to reduce the memory used


Set parameter Username
Set parameter LogFile to value "C:\Users\ghadi\Downloads\Discrete Optimization Project - Gurobi- Instance 9\Discrete Optimization Project - Instance 9-Log File.log"
Academic license - for non-commercial use only - expires 2024-07-07
Set parameter Threads to value 1
Set parameter Presolve to value 1
Set parameter Method to value 0
Set parameter TimeLimit to value 1200
Set parameter PreSparsify to value 1


### Defining Gurobi Model

In [14]:
model = gp.Model("Exam_Scheduling", env=env)

### Defining Decision Variable and Auxiliary Variable

In [15]:
#Decision Variable
X = model.addVars(ExamsList, TimeList , vtype=GRB.BINARY, name="X")

In [16]:
# Binary auxiliary variable for controlling conflict
Y = model.addVars(ExamsList, ExamsList, Distance, vtype=GRB.BINARY, name="Y")

###  Defining Constraints 

In [17]:
#Constraint 1
model.addConstrs(UC_Matrix[i, j] * (X[i, k] + X[j, k]) <= 1 for i in ExamsList for j in ExamsList for k in TimeList)
model.update()

In [18]:
#Constraint 2
model.addConstrs(sum(X[i, k] for k in TimeList) == 1 for i in ExamsList for k in TimeList)
model.update()

In [19]:
#Constraint C3
model.addConstrs((X[i, k] + X[j, k + d]) <= Y[i, j, d] + 1 for i in ExamsList for j in ExamsList for k in TimeList for d in Distance if (k + d) in TimeList and UC_Matrix[i, j] != 0)
model.update()

### Defining Objective Function

In [20]:
# Create the objective expression using a loop
objective_expr = 0
for i in ExamsList:
    for j in ExamsList:
        for d in Distance:
            objective_expr += Y[i, j, d] * WC_Matrix[i, j] * ((2 ** (5 - d))/2726)

# Set up the objective function for minimization
model.setObjective(objective_expr, GRB.MINIMIZE)

In [21]:
# Time limit for the model
model.Params.TimeLimit = 1200

In [22]:
model.update()

In [23]:
model.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i5-1235U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 1 threads

Optimize a model with 3299406 rows, 732663 columns and 2488626 nonzeros
Model fingerprint: 0x7be1b776
Variable types: 0 continuous, 732663 integer (732663 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-04, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 2603033 rows and 680531 columns (presolve time = 5s) ...
Presolve removed 2603061 rows and 680531 columns (presolve time = 10s) ...
Sparsify removed 354 nonzeros (0%)
Presolve removed 2603061 rows and 680501 columns
Presolve time: 10.93s
Presolved: 696345 rows, 52162 columns, 2146213 nonzeros
Variable types: 0 continuous, 52162 integer (52161 binary)
Root relaxation presolved: 52162 rows, 748507 columns, 2198375 nonzeros


Root simplex log

In [24]:
if model.Status == gp.GRB.TIME_LIMIT:

    # Extend the time limit
    extended_time_limit =1200 # Set the extended time limit in seconds
    model.setParam('TimeLimit', extended_time_limit)

    # Continue the optimization
    model.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i5-1235U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 1 threads

Optimize a model with 3299406 rows, 732663 columns and 2488626 nonzeros
Model fingerprint: 0x7be1b776
Variable types: 0 continuous, 732663 integer (732663 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-04, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolved: 696345 rows, 52162 columns, 2146213 nonzeros

Continuing optimization...

     0     0    0.00000    0 1281          -    0.00000      -     - 1258s
     0     0    0.00000    0 1342          -    0.00000      -     - 1362s
     0     0    0.00000    0 1342          -    0.00000      -     - 1428s
H    0     0                      19.8360235    0.00000   100%     - 1619s
H    0     0                      19.7377109    0.00000   100%     - 1633s


In [25]:
if model.Status == GRB.OPTIMAL:
    print("Optimization was successful and an optimal solution was found.")
elif model.Status == GRB.INFEASIBLE:
    print("The model is infeasible - no feasible solution exists.")
elif model.Status == GRB.UNBOUNDED:
    print("The model is unbounded - the objective can be improved without limit.")
print("Objective Value:", model.objVal)

Objective Value: 19.7201027146001


In [26]:
print("Objective Value:", model.objVal)

Objective Value: 19.7201027146001


### Time Slots and Exams in Each Time Slot

In [27]:
# Scheduled-Exams in LIST
scheduled_exams = []
# Print the values of decision variable X
for exam in ExamsList:
    for time in TimeList:
        if X[exam, time].x > 0.5:
            scheduled_exams.append((exam, time))

In [28]:
# Create a dictionary to store exams for each time slot
Exams_in_TimeSlots = {}

for exam, time in scheduled_exams:
    if time not in Exams_in_TimeSlots:
        Exams_in_TimeSlots[time] = []
    Exams_in_TimeSlots[time].append(exam)

# Print the organized structure
for time_slot, exams in sorted(Exams_in_TimeSlots.items()):
    print(f"Time Slot {time_slot}: Exams {exams}")

Time Slot 0: Exams [33, 58, 74, 80, 87, 94, 98, 103, 109, 112, 124, 130, 181, 195, 202, 204, 212, 214, 215, 221, 252, 255, 262, 276, 280, 288, 293, 304, 324, 326, 348, 357]
Time Slot 1: Exams [2, 6, 35, 44, 72, 84, 85, 110, 114, 149, 159, 164, 172, 174, 176, 196, 213, 222, 235, 246, 253, 258, 274, 277, 284, 297, 310, 313, 331, 338, 339, 373, 378, 380]
Time Slot 2: Exams [8, 41, 63, 75, 102, 111, 126, 140, 198, 209, 238, 299, 300, 305, 333, 349, 356, 376, 377]
Time Slot 3: Exams [16, 48, 50, 53, 65, 82, 91, 92, 101, 118, 119, 135, 166, 191, 226, 237, 256, 265, 267, 282, 303, 317, 353]
Time Slot 4: Exams [24, 32, 60, 93, 115, 144, 187, 194, 205, 227, 229, 243, 263, 270, 301, 308, 318, 332, 345, 352, 361, 379]
Time Slot 5: Exams [17, 20, 31, 42, 46, 56, 71, 146, 156, 192, 242, 281, 286, 296, 321]
Time Slot 6: Exams [1, 19, 22, 23, 40, 68, 78, 128, 141, 142, 153, 207, 266, 290, 294, 315, 337, 351]
Time Slot 7: Exams [11, 15, 37, 59, 95, 97, 137, 138, 147, 162, 169, 177, 180, 231, 268, 272,

In [29]:
# Sorting the Dic According to Time Slot It Will be Sorted By Keys 
Exams_in_TimeSlots= dict(sorted(Exams_in_TimeSlots.items()))

### Chcking whether there is Conflict Between the Exams In Each Time Slots

In [30]:
#All pairs of Time Tables exams related to Each time slot-More General
All_pairs_in_TimeSlots_Exams = []

for time_slot, exams in Exams_in_TimeSlots.items():
    exam_pairs = [(exams[i], exams[j]) for i in range(len(exams)) for j in range(i + 1, len(exams))]
    All_pairs_in_TimeSlots_Exams.extend(exam_pairs)

In [31]:
#Conflicting Exams - The code is Correct
List_ConflictExams=[]
for i in ExamsList:
    for j in ExamsList:
        if UC_Matrix[i,j] > 0:
            List_ConflictExams.append((i,j))

In [32]:
# To find Whether There is Intersection Between This Lists or Not

Set_All_pairs_in_TimeSlots_Exams=set(All_pairs_in_TimeSlots_Exams)
Set_List_ConflictExams=set(List_ConflictExams)
Common_elements = Set_All_pairs_in_TimeSlots_Exams.intersection(List_ConflictExams)
# Find common elements
if len (Common_elements) == 0:
    print('The Solution is Feasible Meaning that There is No Conflicting Exam in a All Time Slots')
else:
    print('The Solution is Not Feasible Meaning that There is Conflicting Exam in Time Slots')
    


# Print the common elements
#print("Common Elements:", Common_elements)

The Solution is Feasible Meaning that There is No Conflicting Exam in a All Time Slots


### Necessary OutPuts 

In [33]:
output_folder = r"C:\Users\ghadi\Downloads\Discrete Optimization Project - Gurobi- Instance 9"


output_file_path = os.path.join(output_folder, "Discrete Optimization Project- Instance 9 .sol")

# Specify the file name only for model.write()
model.write("Discrete Optimization Project 2023.sol")

with open(output_file_path, 'w') as f:
    # your code here
    
    # Convert Exams_in_TimeSlots to a string before writing
    exams_string = ", ".join(map(str, TimeList))
    f.write(f"Exams Time Slots are: {exams_string}\n")
    
    f.write('\n\n')
    
    if len (Common_elements) == 0:
        f.write('The Solution is Feasible Meaning that There is No Conflicting Exam in  All Time Slots')
    else:
        f.write('The Solution is Not Feasible Meaning that There is Conflicting Exam in Time Slots')
        
    
    f.write('\n\n')    
        
    
    for time_slot, exams in sorted(Exams_in_TimeSlots.items()):
        f.write(f"Time Slot {time_slot}: Exams {exams}\n")
        
    f.write('\n\n') 
    
    f.write(f"The Objected Value - Penalty: {model.objVal}\n")
    f.write(f"The Benchmark is = 9.818\n")
    